In [2]:
from IPython.display import display, Markdown, HTML
import pandas as pd
import numpy as np

import fastf1 as ff1

ff1.Cache.enable_cache("./data/cache")

YEAR = 2022
RACE = "Austria"

session = ff1.get_session(YEAR, RACE, "Race")
session.load(telemetry=True, laps=True, messages=True, weather=False)


core           INFO 	Loading data for Austrian Grand Prix - Race [v2.2.9]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
api            INFO 	Using cached data for session_status_data
api            INFO 	Using cached data for track_status_data
api            INFO 	Using cached data for car_data
api            INFO 	Using cached data for position_data
api            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['16', '1', '44', '63', '31', '47', '4', '20', '3', '14', '77', '23', '18', '24', '10', '22', '5', '55', '6', '11']


In [3]:
def get_test_stuff():
    with open("./notebooks/test.txt") as f:
        lines = f.readlines()

    woo = {}
    for l in lines:
        parts = l.strip().split(" ")
        lap_number = parts[1]
        driver_name = parts[2]
        passed_name = parts[4]
        position = parts[-1]

        driver = drivers[drivers["LastName"] == driver_name].iloc[0]
        passed = drivers[drivers["LastName"] == passed_name].iloc[0]
        driver_number = driver["DriverNumber"]

        if driver_number not in woo:
            woo[driver_number] = []
        woo[driver_number].append((lap_number, passed["Code"], position))


display(get_test_stuff())


NameError: name 'drivers' is not defined

In [140]:
from scrape.core import get_drivers
from scrape.core.driver import add_driver_code
from scrape.race import get_driver_overtakes
from scrape.race.timing import get_timing_data, add_lap_number_to_timing_data, merge_leader_lap
from scrape.race.overtake import (
    __get_position_changes,
    __get_driver_position_changes,
)

DRIVER = "22"

# TODO: DriverCode should just be named Driver (like ff1.core.Lap)

drivers = get_drivers(YEAR)
timing = get_timing_data(session)

driver_laps = session.laps.pick_driver(DRIVER)
position_changes = __get_position_changes(session.laps, timing)

overtakes = get_driver_overtakes(session, DRIVER, timing, position_changes)
overtakes = add_driver_code(overtakes, drivers)
overtakes = add_driver_code(overtakes, drivers, suffix="Behind")
overtakes = add_driver_code(overtakes, drivers, suffix="Ahead")
overtakes["LapNumber"] = overtakes["LapNumber"].astype("int64")
overtakes = overtakes.sort_values(by="LapNumber")
overtakes = overtakes[
    ["Time", "LapNumber", "PassingStatus", "Code", "Position", "CodeBehind", "CodeAhead"]
]
overtakes_filtered = overtakes[overtakes["PassingStatus"].isnull()]
display(overtakes_filtered)

drv_position_changes = position_changes.query(f'DriverNumber == "{DRIVER}"').reset_index(drop=True)
drv_position_changes = add_driver_code(drv_position_changes, drivers)
drv_position_changes = add_driver_code(drv_position_changes, drivers, suffix="Behind")
drv_position_changes = add_driver_code(drv_position_changes, drivers, suffix="Ahead")
drv_position_changes = add_driver_code(drv_position_changes, drivers, suffix="Passed")
# drv_position_changes["Time"] = position_changes["Time"].astype('timedelta64[s]')
drv_position_changes = drv_position_changes[
    [
        "Time",
        "LapNumber",
        "Code",
        "CodePassed",
        "Position",
        "PositionBefore",
        "PositionGained",
        "CodeBehind",
        "CodeAhead",
    ]
]

from scrape.data import df_timedelta_to_string

display(
    merge_leader_lap(driver_laps, session.laps, timing)[
        [
            "Time",
            "LapTime",
            "LapNumber",
            "LapLeader",
            "LapStartTime",
            "PitInTime",
            "PitOutTime",
            "Sector1Time",
            "Sector2Time",
            "Sector3Time",
            "Sector1SessionTime",
            "Sector2SessionTime",
            "Sector3SessionTime",
        ]
    ]
    .rename(
        {
            "Sector1SessionTime": "S1SessTime",
            "Sector2SessionTime": "S2SessTime",
            "Sector3SessionTime": "S3SessTime",
        }
    )
    .apply(df_timedelta_to_string, axis=1)
    .astype(str)
    .iloc[45:50]
)

print("timing data")
display(timing.query(f'DriverNumber == "{DRIVER}"').reset_index(drop=True)[1000:1050])

display(drv_position_changes[-15:])


api            INFO 	Using cached data for timing_data


,Time,LapNumber,PassingStatus,Code,Position,CodeBehind,CodeAhead
1,0 days 01:04:00.967000,2,NaN,TSU,14,ZHO,ALB
6,0 days 01:17:29.953000,13,NaN,TSU,10,ZHO,STR
12,0 days 01:40:19.316000,32,NaN,TSU,16,LAT,STR


,Time,LapTime,LapNumber,LapLeader,LapStartTime,PitInTime,PitOutTime,Sector1Time,Sector2Time,Sector3Time,Sector1SessionTime,Sector2SessionTime,Sector3SessionTime
1100,1:58:03.579,01:15.742,46,48,1:56:47.837,1:58:00.254,NaT,17.710,32.629,25.403,1:57:05.547,1:57:38.176,1:58:03.579
1101,1:59:32.307,01:28.728,47,49,1:58:03.579,NaT,1:58:21.708,35.012,32.084,21.632,1:58:38.591,1:59:10.675,1:59:32.307
1102,2:00:43.008,01:10.701,48,50,1:59:32.307,NaT,NaT,17.637,31.550,21.514,1:59:49.944,2:00:21.494,2:00:43.008
1103,2:01:53.667,01:10.659,49,51,2:00:43.008,NaT,NaT,17.607,31.473,21.579,2:01:00.615,2:01:32.088,2:01:53.667
1104,2:03:03.940,01:10.273,50,52,2:01:53.667,NaT,NaT,17.506,31.304,21.463,2:02:11.173,2:02:42.477,2:03:03.940


timing data


,Time,DriverNumber,Position,GapToLeader,IntervalToPositionAhead
1000,0 days 02:01:35.730000,22,16,1 L,+1.585
1001,0 days 02:01:40.941000,22,16,1 L,+1.616
1002,0 days 02:01:43.344000,22,16,1 L,+1.590
1003,0 days 02:01:46.434000,22,16,1 L,+1.643
1004,0 days 02:01:47.831000,22,16,1 L,+1.668
1005,0 days 02:01:50.703000,22,16,1 L,+1.659
1006,0 days 02:01:53.635000,22,16,1 L,+1.686
1007,0 days 02:01:57.145000,22,16,1 L,+1.749
1008,0 days 02:01:58.696000,22,16,1 L,+1.815
1009,0 days 02:02:02.668000,22,16,1 L,+1.771


,Time,LapNumber,Code,CodePassed,Position,PositionBefore,PositionGained,CodeBehind,CodeAhead
37,0 days 01:52:03.126000,42,TSU,ALB,14,15,1.0,ALB,RUS
38,0 days 01:54:33.679000,44,TSU,NOR,13,14,1.0,NOR,MAG
39,0 days 01:55:39.761000,45,TSU,RIC,12,13,1.0,RIC,ZHO
40,0 days 01:57:18.333000,46,TSU,NaN,13,12,-1.0,RIC,NOR
41,0 days 01:57:53.750000,46,TSU,NaN,14,13,-1.0,ALB,RIC
42,0 days 01:58:02.336000,46,TSU,NaN,15,14,-1.0,BOT,ALB
43,0 days 01:58:07.106000,47,TSU,NaN,16,15,-1.0,GAS,ZHO
44,0 days 01:58:16.179000,47,TSU,NaN,17,16,-1.0,VET,ZHO
45,0 days 01:58:21.616000,47,TSU,ZHO,16,17,1.0,ZHO,GAS
46,0 days 02:00:22.018000,48,TSU,VET,15,16,1.0,VET,GAS
